<a href="https://colab.research.google.com/github/aadegokemuideen/dino/blob/master/CoreLearningAlgorithmClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This file focus on classification problem: The process of classiying data points into different classes. In this file I used tensorflow 2.x. I worked on Irish Data set

In [109]:
'''
from DinoPreprocessor.test3 import sigmoid
print(sigmoid(1))'''

%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

In [110]:
csvColumnNames = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
Species = ["Setosa", "Versicolor", "Virginica"]


# load irish data set. save the data set as irish_training.csv in keras/datasets/irish_training.csv
# Similarly for "irish_test.csv"
path_train = tf.keras.utils.get_file("irish_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
path_test = tf.keras.utils.get_file("irish_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")

In [111]:
print(path_train) # print this to see the name of the path
train = pd.read_csv(path_train,names= csvColumnNames, header =0)
test = pd.read_csv(path_test, names = csvColumnNames, header =0)


/root/.keras/datasets/irish_training.csv


In [112]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [113]:
# let pop the Species column up to serve as our teacher or output example

train_y = train.pop("Species")
test_y = test.pop("Species")

# The species column is now gone
train.head() # The species column is now gone

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [114]:
def input_fn(features, labels, training = True, batch_size=256):
  #Convert the input to a Dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  #shuffle and repeat if the you are in the training mode
  if training:
    dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size) # return dataset with given batchsize

In [115]:
# feature columns describe how to use the input

my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))

print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]




```
# This is formatted as code
```

# Let create a classification model

In [116]:
#Build a DNN with 2 hidden layer with 30 node and 10 hiddee nodes
classifier = tf.estimator.DNNClassifier(feature_columns = my_feature_columns,
                                        #Two hidden layers of 30 and 10 nodes resepectively
                                        hidden_units =[30,10],
                                        # The model must choose between 3 classes
                                        n_classes = 3)

classifier.train(input_fn = lambda: input_fn(train, train_y, training=True), steps = 5000)
#Here we utilize a lambda function to avoid creating an inner function like the previous model in used in titanic data set


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpj6noxemv', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. 

# Now let evaluate our model. 

In [117]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
print(eval_result)
print('\nTest data set accuracy: {accuracy:.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-11-28T14:10:41Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpj6noxemv/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.20192s
INFO:tensorflow:Finished evaluation at 2020-11-28-14:10:41
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.7583333, average

## Code block for model prediction accuracy. Here is the function for the code block

In [118]:
def input_fn(features, batch_size=256):
  #convert the input feature to a Dataset without lables

  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)
features =["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"]
predict = {}

print("please type the numeric values as prompted.")

for feature in features:
  valid = True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit(): valid = False
  
  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
  class_id = pred_dict["class_ids"][0]
  probability = pred_dict["probabilities"][class_id]

  print("prediction is {} ({:.1f}%)".format(Species[class_id], 100 * probability))


please type the numeric values as prompted.
SepalLength: 1.3
SepalWidth: 1.56
PetalLength: 1.32
PetalWidth: 1.1
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpj6noxemv/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
prediction is Virginica (36.6%)
